In [ ]:
#!/usr/bin/env python

h_id='asdf'
h_key='asdf'

import json.decoder
from datetime import date
from requests_hawk import HawkAuth
import requests


base_url = 'https://app.absence.io/api/v2'
hawk = HawkAuth(id=h_id, key=h_key)
start_date = date(2018, 1, 1)
paging = 180


def main():
    debug_info = []
    url = '{}/{}'.format(base_url, 'absences')
    data = {
        "relations": ["assignedToId", "reasonId"],
        "filter": {
            'start': {"$gte": str(start_date)}
        },
        "responseModel": "Calendar",
        "sortBy": {"startDateTime": 1},
        "limit": paging
    }
    has_next = True
    result = {}
    skip = 0
    while has_next:
        print('---')
        data['skip'] = skip
        print(data)
        response = requests.post(url, json=data, auth=hawk)
        try:
            json_data = response.json()
        except json.decoder.JSONDecodeError as e:
            print(e)
            print(response)
        if response.status_code != 200:
            has_next = False
        limit = json_data['limit']
        count = json_data['count']
        skip = json_data['skip'] + count
        if count < limit:
            has_next = False
        for response_data in json_data['data']:
            absence_id = response_data['_id']
            if absence_id in result:
                print('absence_id {} already in result !'.format(absence_id))
                i = 0
                for d in debug_info:
                    if d == absence_id:
                        print('\twas at position {}'.format(i+1))
                    i=i+1
            result[absence_id] = response_data
            debug_info.append(absence_id)
        print('len(result): {}'.format(len(result)))
    return result


if __name__ == '__main__':
    main()
